In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_model_optimization as tfmot

In [5]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs=4,
  validation_split=0.1,
)

Epoch 1/4
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2908 - accuracy: 0.9195 - val_loss: 0.1170 - val_accuracy: 0.9667
Epoch 2/4
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1168 - accuracy: 0.9662 - val_loss: 0.0854 - val_accuracy: 0.9767
Epoch 3/4
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0849 - accuracy: 0.9752 - val_loss: 0.0818 - val_accuracy: 0.9782
Epoch 4/4
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0704 - accuracy: 0.9793 - val_loss: 0.0638 - val_accuracy: 0.9827


In [7]:
# 가지치기를 위한 TensorFlow 모듈 불러오기
from tensorflow_model_optimization.sparsity import keras as sparsity

# 가지치기 모델 생성
pruning_params = {
    'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.2,
                                                 final_sparsity=0.8,
                                                 begin_step=0,
                                                 end_step=1000,
                                                 frequency=100)
}

# 가지치기 콜백 생성
pruning_callbacks = [
    sparsity.UpdatePruningStep()
]

# 가지치기 모델 컴파일
pruned_model = sparsity.prune_low_magnitude(model, **pruning_params)

# 가지치기 모델 훈련 및 평가
pruned_model.compile(optimizer='adam',
                     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                     metrics=['accuracy'])

pruned_model.fit(
  train_images,
  train_labels,
  epochs=4,
  validation_split=0.1,
  callbacks=pruning_callbacks  # 가지치기 콜백 추가
)

Epoch 1/4
1688/1688 [==============================] - 8s 4ms/step - loss: 0.1287 - accuracy: 0.9655 - val_loss: 0.1001 - val_accuracy: 0.9732
Epoch 2/4
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1012 - accuracy: 0.9705 - val_loss: 0.0839 - val_accuracy: 0.9783
Epoch 3/4
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0877 - accuracy: 0.9741 - val_loss: 0.0787 - val_accuracy: 0.9783
Epoch 4/4
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0805 - accuracy: 0.9766 - val_loss: 0.0753 - val_accuracy: 0.9788


In [8]:
# 양자화를 위한 TensorFlow 모듈 불러오기
import tensorflow_model_optimization as tfmot

# 양자화 모델 생성
quantize_model = tfmot.quantization.keras.quantize_model(model)

# 양자화 모델 컴파일
quantize_model.compile(optimizer='adam',
                       loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                       metrics=['accuracy'])

# 양자화 모델 훈련 및 평가
quantize_model.fit(
  train_images,
  train_labels,
  epochs=4,
  validation_split=0.1,
)

Epoch 1/4
1688/1688 [==============================] - 8s 4ms/step - loss: 0.0751 - accuracy: 0.9771 - val_loss: 0.0758 - val_accuracy: 0.9797
Epoch 2/4
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0612 - accuracy: 0.9817 - val_loss: 0.0680 - val_accuracy: 0.9783
Epoch 3/4
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0538 - accuracy: 0.9836 - val_loss: 0.0631 - val_accuracy: 0.9832
Epoch 4/4
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0490 - accuracy: 0.9848 - val_loss: 0.0619 - val_accuracy: 0.9823


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot
import time
import tempfile
import zipfile  # 'zipfile' 모듈을 추가로 불러옵니다.

# get_gzipped_model_size 함수 정의
def get_gzipped_model_size(file):
    # Returns size of gzipped model, in bytes.
    _, zipped_file = tempfile.mkstemp('.zip')
    with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
        f.write(file)

    return os.path.getsize(zipped_file)


In [2]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(28, 28)),
    keras.layers.Reshape(target_shape=(28, 28, 1)),
    keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
    train_images,
    train_labels,
    epochs=1,
    validation_split=0.1,
)

# 원본 모델 평가
print("원본 모델 평가:")
original_loss, original_accuracy = model.evaluate(test_images, test_labels)
print(f"원본 모델 정확성: {original_accuracy}")


1688/1688 [==============================] - 19s 11ms/step - loss: 0.1436 - accuracy: 0.9561 - val_loss: 0.0615 - val_accuracy: 0.9820
원본 모델 평가:
313/313 [==============================] - 1s 4ms/step - loss: 0.0607 - accuracy: 0.9813
원본 모델 정확성: 0.9812999963760376


In [3]:
# 가지치기 테스트
from tensorflow_model_optimization.sparsity import keras as sparsity

# 가지치기 모델 생성
pruning_params = {
    'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.2,
                                                 final_sparsity=0.8,
                                                 begin_step=0,
                                                 end_step=1000,
                                                 frequency=100)
}

# 가지치기 콜백 생성
pruning_callbacks = [
    sparsity.UpdatePruningStep()
]

# 가지치기 모델 컴파일
pruned_model = sparsity.prune_low_magnitude(model, **pruning_params)

# 가지치기 모델 훈련 및 평가
pruned_model.compile(optimizer='adam',
                     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                     metrics=['accuracy'])

pruned_model.fit(
    train_images,
    train_labels,
    epochs=1,
    validation_split=0.1,
    callbacks=pruning_callbacks
)

print("가지치기 모델 평가:")
pruned_loss, pruned_accuracy = pruned_model.evaluate(test_images, test_labels)
print(f"가지치기 모델 정확성: {pruned_accuracy}")

1688/1688 [==============================] - 21s 11ms/step - loss: 0.0563 - accuracy: 0.9830 - val_loss: 0.0398 - val_accuracy: 0.9892
가지치기 모델 평가:
313/313 [==============================] - 1s 4ms/step - loss: 0.0366 - accuracy: 0.9882
가지치기 모델 정확성: 0.9882000088691711


In [4]:
# 양자화 테스트
import tensorflow_model_optimization as tfmot

# 양자화 모델 생성
quantize_model = tfmot.quantization.keras.quantize_model(model)

# 양자화 모델 컴파일
quantize_model.compile(optimizer='adam',
                       loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                       metrics=['accuracy'])

# 양자화 모델 훈련 및 평가
quantize_model.fit(
    train_images,
    train_labels,
    epochs=1,
    validation_split=0.1,
)

print("양자화 모델 평가:")
quantized_loss, quantized_accuracy = quantize_model.evaluate(test_images, test_labels)
print(f"양자화 모델 정확성: {quantized_accuracy}")

1688/1688 [==============================] - 24s 13ms/step - loss: 0.0478 - accuracy: 0.9852 - val_loss: 0.0505 - val_accuracy: 0.9852
양자화 모델 평가:
313/313 [==============================] - 2s 5ms/step - loss: 0.0491 - accuracy: 0.9850
양자화 모델 정확성: 0.9850000143051147


In [5]:
# 성능 비교
print("성능 비교:")
print(f"원본 모델 정확성: {original_accuracy}")
print(f"가지치기 모델 정확성: {pruned_accuracy}")
print(f"양자화 모델 정확성: {quantized_accuracy}")

# 모델 크기 비교
original_model_size = len(model.to_json())
pruned_model_size = len(pruned_model.to_json())
quantized_model_size = len(quantize_model.to_json())

print(f"원본 모델 크기: {original_model_size} bytes")
print(f"가지치기 모델 크기: {pruned_model_size} bytes")
print(f"양자화 모델 크기: {quantized_model_size} bytes")

print(f"원본 모델 압축 크기: {get_gzipped_model_size(model.to_json())} bytes")
print(f"가지치기 모델 압축 크기: {get_gzipped_model_size(pruned_model.to_json())} bytes")
print(f"양자화 모델 압축 크기: {get_gzipped_model_size(quantize_model.to_json())} bytes")


성능 비교:
원본 모델 정확성: 0.9812999963760376
가지치기 모델 정확성: 0.9882000088691711
양자화 모델 정확성: 0.9850000143051147
원본 모델 크기: 4855 bytes
가지치기 모델 크기: 9753 bytes
양자화 모델 크기: 10514 bytes


FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: '{"class_name": "Sequential", "config": {"name": "sequential", "layers": [{"module": "keras.layers", "class_name": "InputLayer", "config": {"batch_input_shape": [null, 28, 28], "dtype": "float32", "sparse": false, "ragged": false, "name": "input_1"}, "registered_name": null}, {"module": "keras.layers", "class_name": "Reshape", "config": {"name": "reshape", "trainable": true, "dtype": "float32", "target_shape": [28, 28, 1]}, "registered_name": null, "build_config": {"input_shape": [null, 28, 28]}}, {"module": "keras.layers", "class_name": "Conv2D", "config": {"name": "conv2d", "trainable": true, "dtype": "float32", "filters": 32, "kernel_size": [3, 3], "strides": [1, 1], "padding": "valid", "data_format": "channels_last", "dilation_rate": [1, 1], "groups": 1, "activation": "relu", "use_bias": true, "kernel_initializer": {"module": "keras.initializers", "class_name": "GlorotUniform", "config": {"seed": null}, "registered_name": null}, "b

In [9]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot
import time
import tempfile
import zipfile
import os

# get_gzipped_model_size 함수 정의
def get_gzipped_model_size(model):
    _, keras_file = tempfile.mkstemp('.h5')
    tf.keras.models.save_model(model, keras_file, include_optimizer=False)
    
    _, zipped_file = tempfile.mkstemp('.zip')
    with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
        f.write(keras_file)
    
    return os.path.getsize(zipped_file)

# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(28, 28)),
    keras.layers.Reshape(target_shape=(28, 28, 1)),
    keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
    train_images,
    train_labels,
    epochs=1,
    validation_split=0.1,
)

# Save the model
model.save("my_model.h5")

# Load the model
loaded_model = keras.models.load_model("my_model.h5")

# 가지치기 함수 정의
def apply_pruning_to_loaded_model(loaded_model):
    pruning_params = {
        'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
            initial_sparsity=0.2,
            final_sparsity=0.8,
            begin_step=0,
            end_step=1000,
            frequency=100
        )
    }

    pruned_model = tfmot.sparsity.keras.prune_low_magnitude(loaded_model, **pruning_params)
    pruned_model.compile(optimizer='adam',
                         loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                         metrics=['accuracy'])

    return pruned_model

# 가지치기 모델 생성
pruned_loaded_model = apply_pruning_to_loaded_model(loaded_model)

# 가지치기 모델 평가
def evaluate_model(model, test_images, test_labels):
    start_time = time.time()
    loss, accuracy = model.evaluate(test_images, test_labels)
    end_time = time.time()
    inference_time = end_time - start_time
    return loss, accuracy, inference_time

# 테스트 데이터 준비
test_images = test_images.reshape((-1, 28, 28, 1))
test_images = test_images / 255.0

# 가지치기 모델 평가
pruned_loss, pruned_accuracy, pruned_inference_time = evaluate_model(pruned_loaded_model, test_images, test_labels)

print("가지치기 모델 평가 결과:")
print(f"손실: {pruned_loss}")
print(f"정확성: {pruned_accuracy}")
print(f"추론 시간: {pruned_inference_time} 초")

# 양자화 함수 정의
def apply_quantization_to_loaded_model(loaded_model):
    quantize_model = tfmot.quantization.keras.quantize_model(loaded_model)
    quantize_model.compile(optimizer='adam',
                           loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                           metrics=['accuracy'])
    return quantize_model

# 양자화 모델 생성
quantized_loaded_model = apply_quantization_to_loaded_model(loaded_model)

# 양자화 모델 평가
quantized_loss, quantized_accuracy, quantized_inference_time = evaluate_model(quantized_loaded_model, test_images, test_labels)

print("양자화 모델 평가 결과:")
print(f"손실: {quantized_loss}")
print(f"정확성: {quantized_accuracy}")
print(f"추론 시간: {quantized_inference_time} 초")

# 성능 비교
print("성능 비교:")
print("원본 모델 정확성:", model.evaluate(test_images, test_labels)[1])
print("가지치기 모델 정확성:", pruned_accuracy)
print("양자화 모델 정확성:", quantized_accuracy)

# 모델 크기 비교
original_model_size = len(loaded_model.to_json())
pruned_model_size = len(pruned_loaded_model.to_json())
quantized_model_size = len(quantized_loaded_model.to_json())

print("원본 모델 크기:", original_model_size, "bytes")
print("가지치기 모델 크기:", pruned_model_size, "bytes")
print("양자화 모델 크기:", quantized_model_size, "bytes")

# 압축된 모델 크기 비교
original_gzipped_model_size = get_gzipped_model_size(loaded_model)
pruned_gzipped_model_size = get_gzipped_model_size(pruned_loaded_model)
quantized_gzipped_model_size = get_gzipped_model_size(quantized_loaded_model)

print("원본 모델 압축 크기:", original_gzipped_model_size, "bytes")
print("가지치기 모델 압축 크기:", pruned_gzipped_model_size, "bytes")
print("양자화 모델 압축 크기:", quantized_gzipped_model_size, "bytes")


313/313 [==============================] - 2s 4ms/step - loss: 2.3004 - accuracy: 0.1135
가지치기 모델 평가 결과:
손실: 2.3004472255706787
정확성: 0.11349999904632568
추론 시간: 1.5863237380981445 초
313/313 [==============================] - 2s 5ms/step - loss: 2.3033 - accuracy: 0.1135
양자화 모델 평가 결과:
손실: 2.3033483028411865
정확성: 0.11349999904632568
추론 시간: 1.9169261455535889 초
성능 비교:


ValueError: in user code:

    File "c:\Users\Jay\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2066, in test_function  *
        return step_function(self, iterator)
    File "c:\Users\Jay\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2049, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Jay\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2037, in run_step  **
        outputs = model.test_step(data)
    File "c:\Users\Jay\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1917, in test_step
        y_pred = self(x, training=False)
    File "c:\Users\Jay\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None

    ValueError

In [11]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot
import time
import tempfile
import zipfile
import os

# get_gzipped_model_size 함수 정의
def get_gzipped_model_size(model):
    _, keras_file = tempfile.mkstemp('.h5')
    tf.keras.models.save_model(model, keras_file, include_optimizer=False)
    
    _, zipped_file = tempfile.mkstemp('.zip')
    with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
        f.write(keras_file)
    
    return os.path.getsize(zipped_file)

# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(28, 28)),
    keras.layers.Reshape(target_shape=(28, 28, 1)),
    keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
    train_images,
    train_labels,
    epochs=1,
    validation_split=0.1,
)

# Save the model
model.save("my_model.h5")

# Load the model
loaded_model = keras.models.load_model("my_model.h5")

# 가지치기 함수 정의
def apply_pruning_to_loaded_model(loaded_model):
    pruning_params = {
        'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
            initial_sparsity=0.2,
            final_sparsity=0.8,
            begin_step=0,
            end_step=1000,
            frequency=100
        )
    }

    pruned_model = tfmot.sparsity.keras.prune_low_magnitude(loaded_model, **pruning_params)
    pruned_model.compile(optimizer='adam',
                         loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                         metrics=['accuracy'])

    return pruned_model

# 가지치기 모델 생성
pruned_loaded_model = apply_pruning_to_loaded_model(loaded_model)

# 가지치기 모델 평가
_, pruned_accuracy = pruned_loaded_model.evaluate(test_images, test_labels)

print("가지치기 모델 정확성:", pruned_accuracy)

# 양자화 함수 정의
def apply_quantization_to_loaded_model(loaded_model):
    quantize_model = tfmot.quantization.keras.quantize_model(loaded_model)
    quantize_model.compile(optimizer='adam',
                           loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                           metrics=['accuracy'])
    return quantize_model

# 양자화 모델 생성
quantized_loaded_model = apply_quantization_to_loaded_model(loaded_model)

# 양자화 모델 평가
_, quantized_accuracy = quantized_loaded_model.evaluate(test_images, test_labels)

print("양자화 모델 정확성:", quantized_accuracy)

# 성능 비교
print("성능 비교:")
print("원본 모델 정확성:", original_accuracy)
print("가지치기 모델 정확성:", pruned_accuracy)
print("양자화 모델 정확성:", quantized_accuracy)

# 모델 크기 비교
original_model_size = len(loaded_model.to_json())
pruned_model_size = len(pruned_loaded_model.to_json())
quantized_model_size = len(quantized_loaded_model.to_json())

print("원본 모델 크기:", original_model_size, "bytes")
print("가지치기 모델 크기:", pruned_model_size, "bytes")
print("양자화 모델 크기:", quantized_model_size, "bytes")

# 압축된 모델 크기 비교
original_gzipped_model_size = get_gzipped_model_size(loaded_model)
pruned_gzipped_model_size = get_gzipped_model_size(pruned_loaded_model)
quantized_gzipped_model_size = get_gzipped_model_size(quantized_loaded_model)

print("원본 모델 압축 크기:", original_gzipped_model_size, "bytes")
print("가지치기 모델 압축 크기:", pruned_gzipped_model_size, "bytes")
print("양자화 모델 압축 크기:", quantized_gzipped_model_size, "bytes")


313/313 [==============================] - 1s 4ms/step - loss: 0.0531 - accuracy: 0.9823
가지치기 모델 정확성: 0.9822999835014343
313/313 [==============================] - 2s 5ms/step - loss: 2.3015 - accuracy: 0.1135
양자화 모델 정확성: 0.11349999904632568
성능 비교:
원본 모델 정확성: 0.9812999963760376
가지치기 모델 정확성: 0.9822999835014343
양자화 모델 정확성: 0.11349999904632568
원본 모델 크기: 4874 bytes
가지치기 모델 크기: 9789 bytes
양자화 모델 크기: 10552 bytes
원본 모델 압축 크기: 865366 bytes
가지치기 모델 압축 크기: 870030 bytes
양자화 모델 압축 크기: 867916 bytes


C:\Users\Jay\AppData\Local\Temp\ipykernel_21552\1632045003.py:12: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model, keras_file, include_optimizer=False)


In [13]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot
import time
import tempfile
import zipfile
import os

# get_gzipped_model_size 함수 정의
def get_gzipped_model_size(model):
    _, keras_file = tempfile.mkstemp('.h5')
    tf.keras.models.save_model(model, keras_file, include_optimizer=False)
    
    _, zipped_file = tempfile.mkstemp('.zip')
    with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
        f.write(keras_file)
    
    return os.path.getsize(zipped_file)

# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(28, 28)),
    keras.layers.Reshape(target_shape=(28, 28, 1)),
    keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
    train_images,
    train_labels,
    epochs=1,
    validation_split=0.1,
)

# Save the model in HDF5 format
model.save("my_model.h5")

# Load the model
loaded_model = keras.models.load_model("my_model.h5")

# 가지치기 함수 정의
def apply_pruning_to_loaded_model(loaded_model):
    pruning_params = {
        'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
            initial_sparsity=0.2,
            final_sparsity=0.8,
            begin_step=0,
            end_step=1000,
            frequency=100
        )
    }

    pruned_model = tfmot.sparsity.keras.prune_low_magnitude(loaded_model, **pruning_params)
    pruned_model.compile(optimizer='adam',
                         loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                         metrics=['accuracy'])

    return pruned_model

# 가지치기 모델 생성
pruned_model = apply_pruning_to_loaded_model(loaded_model)

# 가지치기 모델 평가
_, pruned_accuracy = pruned_model.evaluate(test_images, test_labels)

print("가지치기 모델 정확성:", pruned_accuracy)

# 양자화 함수 정의
def apply_quantization_to_loaded_model(loaded_model):
    quantize_model = tfmot.quantization.keras.quantize_model(loaded_model)
    quantize_model.compile(optimizer='adam',
                           loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                           metrics=['accuracy'])
    return quantize_model

# 양자화 모델 생성
quantized_model = apply_quantization_to_loaded_model(loaded_model)

# 양자화 모델 평가
_, quantized_accuracy = quantized_model.evaluate(test_images, test_labels)

print("양자화 모델 정확성:", quantized_accuracy)

# 성능 비교
print("성능 비교:")
print("원본 모델 정확성:", original_accuracy)
print("가지치기 모델 정확성:", pruned_accuracy)
print("양자화 모델 정확성:", quantized_accuracy)

# 모델 크기 비교
original_model_size = len(loaded_model.to_json())
pruned_model_size = len(pruned_model.to_json())
quantized_model_size = len(quantized_model.to_json())

print("원본 모델 크기:", original_model_size, "bytes")
print("가지치기 모델 크기:", pruned_model_size, "bytes")
print("양자화 모델 크기:", quantized_model_size, "bytes")

# 압축된 모델 크기 비교
original_gzipped_model_size = get_gzipped_model_size(loaded_model)
pruned_gzipped_model_size = get_gzipped_model_size(pruned_model)
quantized_gzipped_model_size = get_gzipped_model_size(quantized_model)

print("원본 모델 압축 크기:", original_gzipped_model_size, "bytes")
print("가지치기 모델 압축 크기:", pruned_gzipped_model_size, "bytes")
print("양자화 모델 압축 크기:", quantized_gzipped_model_size, "bytes")


313/313 [==============================] - 2s 4ms/step - loss: 0.0569 - accuracy: 0.9806
가지치기 모델 정확성: 0.9805999994277954
313/313 [==============================] - 2s 5ms/step - loss: 2.3145 - accuracy: 0.1135
양자화 모델 정확성: 0.11349999904632568
성능 비교:
원본 모델 정확성: 0.9812999963760376
가지치기 모델 정확성: 0.9805999994277954
양자화 모델 정확성: 0.11349999904632568
원본 모델 크기: 4874 bytes
가지치기 모델 크기: 9789 bytes
양자화 모델 크기: 10552 bytes
원본 모델 압축 크기: 865860 bytes
가지치기 모델 압축 크기: 870478 bytes
양자화 모델 압축 크기: 868354 bytes


C:\Users\Jay\AppData\Local\Temp\ipykernel_21552\2276957639.py:12: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model, keras_file, include_optimizer=False)


In [16]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot
import time
import tempfile
import zipfile
import os

# get_gzipped_model_size 함수 정의
def get_gzipped_model_size(model):
    _, keras_file = tempfile.mkstemp('.h5')
    tf.keras.models.save_model(model, keras_file, include_optimizer=False)
    
    _, zipped_file = tempfile.mkstemp('.zip')
    with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
        f.write(keras_file)
    
    return os.path.getsize(zipped_file)

# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define a more complex CNN model architecture.
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(28, 28)),
    keras.layers.Reshape(target_shape=(28, 28, 1)),
    keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),  # 드롭아웃 레이어 추가
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),  # 드롭아웃 레이어 추가
    keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
    train_images,
    train_labels,
    epochs=1,
    validation_split=0.1,
)

# Save the model in HDF5 format
model.save("my_model.h5")

# Load the model
loaded_model = keras.models.load_model("my_model.h5")

# 가지치기 함수 정의
def apply_pruning_to_loaded_model(loaded_model):
    pruning_params = {
        'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
            initial_sparsity=0.2,
            final_sparsity=0.8,
            begin_step=0,
            end_step=1000,
            frequency=100
        )
    }

    pruned_model = tfmot.sparsity.keras.prune_low_magnitude(loaded_model, **pruning_params)
    pruned_model.compile(optimizer='adam',
                         loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                         metrics=['accuracy'])

    return pruned_model

# 가지치기 모델 생성
pruned_model = apply_pruning_to_loaded_model(loaded_model)

# 가지치기 모델 평가
_, pruned_accuracy = pruned_model.evaluate(test_images, test_labels)

print("가지치기 모델 정확성:", pruned_accuracy)

# 양자화 함수 정의
def apply_quantization_to_loaded_model(loaded_model):
    quantize_model = tfmot.quantization.keras.quantize_model(loaded_model)
    quantize_model.compile(optimizer='adam',
                           loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                           metrics=['accuracy'])
    return quantize_model

# 양자화 모델 생성
quantized_model = apply_quantization_to_loaded_model(loaded_model)

# 양자화 모델 평가
_, quantized_accuracy = quantized_model.evaluate(test_images, test_labels)

print("양자화 모델 정확성:", quantized_accuracy)

# 성능 비교
print("성능 비교:")
print("원본 모델 정확성:", original_accuracy)
print("가지치기 모델 정확성:", pruned_accuracy)
print("양자화 모델 정확성:", quantized_accuracy)

# 모델 크기 비교
original_model_size = len(loaded_model.to_json())
pruned_model_size = len(pruned_model.to_json())
quantized_model_size = len(quantized_model.to_json())

print("원본 모델 크기:", original_model_size, "bytes")
print("가지치기 모델 크기:", pruned_model_size, "bytes")
print("양자화 모델 크기:", quantized_model_size, "bytes")

# 압축된 모델 크기 비교
original_gzipped_model_size = get_gzipped_model_size(loaded_model)
pruned_gzipped_model_size = get_gzipped_model_size(pruned_model)
quantized_gzipped_model_size = get_gzipped_model_size(quantized_model)

print("원본 모델 압축 크기:", original_gzipped_model_size, "bytes")
print("가지치기 모델 압축 크기:", pruned_gzipped_model_size, "bytes")
print("양자화 모델 압축 크기:", quantized_gzipped_model_size, "bytes")


313/313 [==============================] - 2s 4ms/step - loss: 0.0549 - accuracy: 0.9819
가지치기 모델 정확성: 0.9818999767303467
313/313 [==============================] - 2s 5ms/step - loss: 2.3066 - accuracy: 0.0974
양자화 모델 정확성: 0.09740000218153
성능 비교:
원본 모델 정확성: 0.9812999963760376
가지치기 모델 정확성: 0.9818999767303467
양자화 모델 정확성: 0.09740000218153
원본 모델 크기: 5363 bytes
가지치기 모델 크기: 11359 bytes
양자화 모델 크기: 11934 bytes


C:\Users\Jay\AppData\Local\Temp\ipykernel_21552\79366680.py:12: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model, keras_file, include_optimizer=False)


원본 모델 압축 크기: 865923 bytes
가지치기 모델 압축 크기: 870878 bytes
양자화 모델 압축 크기: 868652 bytes


In [17]:
import numpy as np

# 원본 모델의 가중치
original_weights = [layer.get_weights() for layer in loaded_model.layers]

# 가지치기 모델의 가중치
pruned_weights = [layer.get_weights() for layer in pruned_model.layers]

# 양자화 모델의 가중치
quantized_weights = [layer.get_weights() for layer in quantized_model.layers]

# 레이어별로 가중치가 어떻게 바뀌었는지 확인
for i, (original_layer, pruned_layer, quantized_layer) in enumerate(zip(original_weights, pruned_weights, quantized_weights)):
    if len(original_layer) > 0:  # 가중치가 있는 레이어만 비교
        original_shape = np.array(original_layer[0]).shape
        pruned_shape = np.array(pruned_layer[0]).shape
        quantized_shape = np.array(quantized_layer[0]).shape

        print(f"Layer {i+1}:")
        print(f"Original Shape: {original_shape}")
        print(f"Pruned Shape: {pruned_shape}")
        print(f"Quantized Shape: {quantized_shape}")

        # 원본 가중치와 가지치기 모델, 양자화 모델 간의 차이 계산
        weight_diff_pruned = np.abs(np.array(original_layer[0]) - np.array(pruned_layer[0]))
        weight_diff_quantized = np.abs(np.array(original_layer[0]) - np.array(quantized_layer[0]))

        print(f"Weight Difference (Original vs. Pruned): {np.mean(weight_diff_pruned)}")
        print(f"Weight Difference (Original vs. Quantized): {np.mean(weight_diff_quantized)}")
        print("\n")


Layer 2:
Original Shape: (3, 3, 1, 32)
Pruned Shape: (3, 3, 1, 32)
Quantized Shape: ()
Weight Difference (Original vs. Pruned): 0.0
Weight Difference (Original vs. Quantized): 1.011624813079834


Layer 4:
Original Shape: (3, 3, 32, 64)
Pruned Shape: (3, 3, 32, 64)
Quantized Shape: ()
Weight Difference (Original vs. Pruned): 0.0
Weight Difference (Original vs. Quantized): 0.9835658669471741


Layer 7:
Original Shape: (1600, 128)
Pruned Shape: (1600, 128)
Quantized Shape: ()
Weight Difference (Original vs. Pruned): 0.0
Weight Difference (Original vs. Quantized): 0.9931374192237854


Layer 9:
Original Shape: (128, 64)
Pruned Shape: (128, 64)
Quantized Shape: ()
Weight Difference (Original vs. Pruned): 0.0
Weight Difference (Original vs. Quantized): 1.002368688583374


Layer 11:
Original Shape: (64, 10)
Pruned Shape: (64, 10)
Quantized Shape: ()
Weight Difference (Original vs. Pruned): 0.0
Weight Difference (Original vs. Quantized): 0.9837929010391235




In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot
import time
import tempfile
import zipfile
import os
import numpy as np

# get_gzipped_model_size 함수 정의
def get_gzipped_model_size(model):
    _, keras_file = tempfile.mkstemp('.h5')
    tf.keras.models.save_model(model, keras_file, include_optimizer=False)
    
    _, zipped_file = tempfile.mkstemp('.zip')
    with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
        f.write(keras_file)
    
    return os.path.getsize(zipped_file)

In [2]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(28, 28)),
    keras.layers.Reshape(target_shape=(28, 28, 1)),
    keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
    train_images,
    train_labels,
    epochs=1,
    validation_split=0.1,
)


1688/1688 [==============================] - 18s 10ms/step - loss: 0.1450 - accuracy: 0.9562 - val_loss: 0.0501 - val_accuracy: 0.9848


In [3]:
# 가지치기 함수 정의
def apply_pruning_to_loaded_model(loaded_model):
    pruning_params = {
        'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
            initial_sparsity=0.2,
            final_sparsity=0.8,
            begin_step=0,
            end_step=1000,
            frequency=100
        )
    }

    pruned_model = tfmot.sparsity.keras.prune_low_magnitude(loaded_model, **pruning_params)
    pruned_model.compile(optimizer='adam',
                         loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                         metrics=['accuracy'])

    return pruned_model

# 가지치기 모델 생성
loaded_model = keras.models.load_model("my_model.h5")
pruned_model = apply_pruning_to_loaded_model(loaded_model)

# 가지치기 모델의 가지치기 제거
final_pruned_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

In [5]:
# 양자화 함수 정의
def apply_quantization_to_loaded_model(loaded_model):
    quantize_model = tfmot.quantization.keras.quantize_model(loaded_model)
    quantize_model.compile(optimizer='adam',
                           loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                           metrics=['accuracy'])
    return quantize_model

# 양자화 모델 생성
quantized_model = apply_quantization_to_loaded_model(final_pruned_model)

# TFLite 변환 (가지치기 모델을 양자화 모델로 변환)
converter = tf.lite.TFLiteConverter.from_keras_model(quantized_model)
quantized_tflite_model = converter.convert()

# TFLite 모델 저장
with open("quantized_model.tflite", "wb") as f:
    f.write(quantized_tflite_model)

# 양자화 모델 평가
_, quantized_accuracy = quantized_model.evaluate(test_images, test_labels)

print("양자화 모델 정확성:", quantized_accuracy)

INFO:tensorflow:Assets written to: C:\Users\Jay\AppData\Local\Temp\tmpb_ezif9i\assets


INFO:tensorflow:Assets written to: C:\Users\Jay\AppData\Local\Temp\tmpb_ezif9i\assets


313/313 [==============================] - 2s 5ms/step - loss: 2.3066 - accuracy: 0.0974
양자화 모델 정확성: 0.09740000218153


In [6]:
# 성능 비교
print("성능 비교:")
print("원본 모델 정확성:", original_accuracy)
print("가지치기 모델 정확성:", pruned_accuracy)
print("양자화 모델 정확성:", quantized_accuracy)

# 모델 크기 비교
original_model_size = len(loaded_model.to_json())
pruned_model_size = len(pruned_model.to_json())
quantized_model_size = len(quantized_model.to_json())

print("원본 모델 크기:", original_model_size, "bytes")
print("가지치기 모델 크기:", pruned_model_size, "bytes")
print("양자화 모델 크기:", quantized_model_size, "bytes")

# 압축된 모델 크기 비교
original_gzipped_model_size = get_gzipped_model_size(loaded_model)
pruned_gzipped_model_size = get_gzipped_model_size(pruned_model)
quantized_gzipped_model_size = get_gzipped_model_size(quantized_model)

print("원본 모델 압축 크기:", original_gzipped_model_size, "bytes")
print("가지치기 모델 압축 크기:", pruned_gzipped_model_size, "bytes")
print("양자화 모델 압축 크기:", quantized_gzipped_model_size, "bytes")

성능 비교:


NameError: name 'original_accuracy' is not defined

In [7]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot
import time
import tempfile
import zipfile
import os
import numpy as np

# get_gzipped_model_size 함수 정의
def get_gzipped_model_size(model):
    _, keras_file = tempfile.mkstemp('.h5')
    tf.keras.models.save_model(model, keras_file, include_optimizer=False)
    
    _, zipped_file = tempfile.mkstemp('.zip')
    with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
        f.write(keras_file)
    
    return os.path.getsize(zipped_file)

# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(28, 28)),
    keras.layers.Reshape(target_shape=(28, 28, 1)),
    keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
    train_images,
    train_labels,
    epochs=1,
    validation_split=0.1,
)

# Save the model in HDF5 format
model.save("my_model.h5")

# Load the model
loaded_model = keras.models.load_model("my_model.h5")

# 가지치기 함수 정의
def apply_pruning_to_loaded_model(loaded_model):
    pruning_params = {
        'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
            initial_sparsity=0.2,
            final_sparsity=0.8,
            begin_step=0,
            end_step=1000,
            frequency=100
        )
    }

    pruned_model = tfmot.sparsity.keras.prune_low_magnitude(loaded_model, **pruning_params)
    pruned_model.compile(optimizer='adam',
                         loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                         metrics=['accuracy'])

    return pruned_model

# 가지치기 모델 생성
loaded_model = keras.models.load_model("my_model.h5")
pruned_model = apply_pruning_to_loaded_model(loaded_model)

# 가지치기 모델의 가지치기 제거
final_pruned_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

# 양자화 함수 정의
def apply_quantization_to_loaded_model(loaded_model):
    quantize_model = tfmot.quantization.keras.quantize_model(loaded_model)
    quantize_model.compile(optimizer='adam',
                           loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                           metrics=['accuracy'])
    return quantize_model

# 양자화 모델 생성
quantized_model = apply_quantization_to_loaded_model(final_pruned_model)

# TFLite 변환 (가지치기 모델을 양자화 모델로 변환)
converter = tf.lite.TFLiteConverter.from_keras_model(quantized_model)
quantized_tflite_model = converter.convert()

# TFLite 모델 저장
with open("quantized_model.tflite", "wb") as f:
    f.write(quantized_tflite_model)

# 양자화 모델 평가
_, quantized_accuracy = quantized_model.evaluate(test_images, test_labels)

print("양자화 모델 정확성:", quantized_accuracy)

# 원본 모델 평가
_, original_accuracy = loaded_model.evaluate(test_images, test_labels)
print("원본 모델 정확성:", original_accuracy)

# 성능 비교
print("성능 비교:")
print("가지치기 모델 정확성:", final_pruned_model_accuracy)
print("양자화 모델 정확성:", quantized_accuracy)

# 모델 크기 비교
original_model_size = len(loaded_model.to_json())
pruned_model_size = len(final_pruned_model.to_json())
quantized_model_size = len(quantized_model.to_json())

print("원본 모델 크기:", original_model_size, "bytes")
print("가지치기 모델 크기:", pruned_model_size, "bytes")
print("양자화 모델 크기:", quantized_model_size, "bytes")

# 압축된 모델 크기 비교
original_gzipped_model_size = get_gzipped_model_size(loaded_model)
pruned_gzipped_model_size = get_gzipped_model_size(final_pruned_model)
quantized_gzipped_model_size = get_gzipped_model_size(quantized_model)

print("원본 모델 압축 크기:", original_gzipped_model_size, "bytes")
print("가지치기 모델 압축 크기:", pruned_gzipped_model_size, "bytes")
print("양자화 모델 압축 크기:", quantized_gzipped_model_size, "bytes")


1688/1688 [==============================] - 18s 10ms/step - loss: 0.1435 - accuracy: 0.9553 - val_loss: 0.0474 - val_accuracy: 0.9868


c:\Users\Jay\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


INFO:tensorflow:Assets written to: C:\Users\Jay\AppData\Local\Temp\tmpd4qcyjz9\assets


INFO:tensorflow:Assets written to: C:\Users\Jay\AppData\Local\Temp\tmpd4qcyjz9\assets


313/313 [==============================] - 2s 5ms/step - loss: 2.3038 - accuracy: 0.1135
양자화 모델 정확성: 0.11349999904632568
313/313 [==============================] - 1s 4ms/step - loss: 0.0517 - accuracy: 0.9837
원본 모델 정확성: 0.9836999773979187
성능 비교:


NameError: name 'final_pruned_model_accuracy' is not defined

In [21]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot
import time
import tempfile
import zipfile
import os
import numpy as np

# get_gzipped_model_size 함수 정의
def get_gzipped_model_size(model):
    _, keras_file = tempfile.mkstemp('.h5')
    tf.keras.models.save_model(model, keras_file, include_optimizer=False)
    
    _, zipped_file = tempfile.mkstemp('.zip')
    with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
        f.write(keras_file)
    
    return os.path.getsize(zipped_file)

# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 and 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture.
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(28, 28)),
    keras.layers.Reshape(target_shape=(28, 28, 1)),
    keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
    train_images,
    train_labels,
    epochs=1,
    validation_split=0.1,
)

# Save the model in HDF5 format
model.save("my_model.h5")

# Load the model
loaded_model = keras.models.load_model("my_model.h5")

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1  # 10% of training set will be used for the validation set.

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                             final_sparsity=0.80,
                                                             begin_step=0,
                                                             end_step=end_step)
}

# Apply pruning to the loaded model
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
model_for_pruning = prune_low_magnitude(loaded_model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
                          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                          metrics=['accuracy'])

logdir = tempfile.mkdtemp()

callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(train_images, train_labels,
                      batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                      callbacks=callbacks)

_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)

print('Pruned test accuracy:', model_for_pruning_accuracy)

# 가지치기 모델의 가지치기 제거
final_pruned_model = tfmot.sparsity.keras.strip_pruning(model_for_pruning)


_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(final_pruned_model, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)


# 양자화 함수 정의
def apply_quantization_to_loaded_model(loaded_model):
    quantize_model = tfmot.quantization.keras.quantize_model(loaded_model)
    quantize_model.compile(optimizer='adam',
                           loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                           metrics=['accuracy'])
    return quantize_model

# 양자화 모델 생성
quantized_model = apply_quantization_to_loaded_model(final_pruned_model)

# TFLite 변환 (가지치기 모델을 양자화 모델로 변환)
converter = tf.lite.TFLiteConverter.from_keras_model(quantized_model)
quantized_tflite_model = converter.convert()

# TFLite 모델 저장
with open("quantized_model.tflite", "wb") as f:
    f.write(quantized_tflite_model)

# 양자화 모델 평가
_, quantized_accuracy = quantized_model.evaluate(test_images, test_labels)

print("양자화 모델 정확성:", quantized_accuracy)

# 원본 모델 평가
_, original_accuracy = loaded_model.evaluate(test_images, test_labels)
print("원본 모델 정확성:", original_accuracy)

# 성능 비교
print("성능 비교:")
print("가지치기 모델 정확성:", model_for_pruning_accuracy)
print("양자화 모델 정확성:", quantized_accuracy)

# 모델 크기 비교
original_model_size = len(loaded_model.to_json())
pruned_model_size = len(pruned_keras_file.to_json())
quantized_model_size = len(quantized_model.to_json())

print("원본 모델 크기:", original_model_size, "bytes")
print("가지치기 모델 크기:", pruned_model_size, "bytes")
print("양자화 모델 크기:", quantized_model_size, "bytes")

# 압축된 모델 크기 비교
original_gzipped_model_size = get_gzipped_model_size(loaded_model)
pruned_gzipped_model_size = get_gzipped_model_size(pruned_keras_file)
quantized_gzipped_model_size = get_gzipped_model_size(quantized_model)

print("원본 모델 압축 크기:", original_gzipped_model_size, "bytes")
print("가지치기 모델 압축 크기:", pruned_gzipped_model_size, "bytes")
print("양자화 모델 압축 크기:", quantized_gzipped_model_size, "bytes")



1688/1688 [==============================] - 16s 9ms/step - loss: 0.1563 - accuracy: 0.9521 - val_loss: 0.0631 - val_accuracy: 0.9803
Epoch 1/2
422/422 [==============================] - 13s 26ms/step - loss: 0.0584 - accuracy: 0.9827 - val_loss: 0.0523 - val_accuracy: 0.9838
Epoch 2/2
422/422 [==============================] - 11s 26ms/step - loss: 0.0410 - accuracy: 0.9880 - val_loss: 0.0347 - val_accuracy: 0.9898
Pruned test accuracy: 0.9891999959945679


C:\Users\Jay\AppData\Local\Temp\ipykernel_31760\2195050560.py:107: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(final_pruned_model, pruned_keras_file, include_optimizer=False)


Saved pruned Keras model to: C:\Users\Jay\AppData\Local\Temp\tmpyxnpoz4a.h5
INFO:tensorflow:Assets written to: C:\Users\Jay\AppData\Local\Temp\tmphazu83x6\assets


INFO:tensorflow:Assets written to: C:\Users\Jay\AppData\Local\Temp\tmphazu83x6\assets


313/313 [==============================] - 2s 4ms/step - loss: 2.3277 - accuracy: 0.1135
양자화 모델 정확성: 0.11349999904632568
313/313 [==============================] - 1s 3ms/step - loss: 0.0339 - accuracy: 0.9892
원본 모델 정확성: 0.9891999959945679
성능 비교:
가지치기 모델 정확성: 0.9891999959945679
양자화 모델 정확성: 0.11349999904632568


AttributeError: 'str' object has no attribute 'to_json'